In [1]:
import json
import pandas as pd
import numpy as np
from emoji import core
import re
import ast
from matplotlib import pyplot as plt

In [2]:
playlists = pd.read_json('../0_data/playlists.json', typ = 'frame', encoding='utf-8')
song_meta = pd.read_json('../0_data/song_meta.json', typ = 'frame', encoding='utf-8') 
genre_gn_all = pd.read_json('../0_data/genre_gn_all.json', typ = 'frame', encoding='utf-8', orient='index')

### 0. genre

In [ ]:
# 컬럼명 지정
genre_gn_all = genre_gn_all.reset_index().rename(columns={'index':'gnr_code', 0:'gnr_name'})
genre_gn_all.head()

# 대분류 장르코드들만 뽑기
gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:]=='00']

# 소분류 장르코드만 뽑기
dtl_gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] != '00']
dtl_gnr_code.rename(columns = {'gnr_code' : 'dtl_gnr_code', 'gnr_name' : 'dtl_gnr_name'}, inplace = True)

# 대분류 장르코드와 소분류 장르코드 각각의 앞자리 네자리 공통코드 추출
gnr_code = gnr_code.assign(join_code = gnr_code['gnr_code'].str[0:4])
dtl_gnr_code = dtl_gnr_code.assign(join_code = dtl_gnr_code['dtl_gnr_code'].str[0:4])

# join_code(네자리 공통코드) 기준으로 Merge
gnr_code_tree = pd.merge(gnr_code, dtl_gnr_code, how = 'left', on = 'join_code')

gnr_code_tree[['gnr_code', 'gnr_name', 'dtl_gnr_code', 'dtl_gnr_name']]
gnr_code_tree

gnr_big = pd.DataFrame(gnr_code.iloc[:, :2])

In [4]:
# song_meta 저장
gnr_big.to_json('../0_data/genre_gn_all_filter.json', orient='records')

### 1. song_meta

> 0. album name이 비어 있는 노래 삭제

In [5]:
song_meta[song_meta['album_name'].isnull()] # album_name 비어 있는 경우 확인

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
143209,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20150522,None,2318271,[726168],사랑바보,"[GN0500, GN0600]",[민은서밴드],143209
291631,"[GN0105, GN0101]",20101108,None,1065178,[28772],빗속에서,[GN0100],[존박],291631
431110,"[GN0509, GN0601, GN0503, GN0606, GN0501]",20200208,None,10385717,[2203064],화면,"[GN0500, GN0600]",[Oowl Hannal (우울 한날)],431110
552088,"[GN0105, GN0601]",20101108,None,1065178,[28263],본능적으로 (Feat. Swings),"[GN0600, GN0100]",[강승윤],552088


In [6]:
song_meta.dropna(subset=['album_name'], inplace=True) # 삭제
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707985 entries, 0 to 707988
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   song_gn_dtl_gnr_basket  707985 non-null  object
 1   issue_date              707985 non-null  int64 
 2   album_name              707985 non-null  object
 3   album_id                707985 non-null  int64 
 4   artist_id_basket        707985 non-null  object
 5   song_name               707985 non-null  object
 6   song_gn_gnr_basket      707985 non-null  object
 7   artist_name_basket      707985 non-null  object
 8   id                      707985 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 54.0+ MB


> 1. 날짜 오류 수정

In [7]:
# 날짜 오류있는 행 확인
song_meta[song_meta['issue_date'] > 20220000] # index : 141185, 448286

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
141185,[GN1801],20220113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],잔잔한 바람이 불어와 (There is a gentle breeze),[GN1800],[무드홀릭 (Moodholic)],141185
448286,[GN1801],20220113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],여수 바다 (Yeosu sea),[GN1800],[무드홀릭 (Moodholic)],448286


In [8]:
indices_to_update = [141185, 448286]
new_value = 20200113 # 알맞는 발매날짜
# 특정 행의 'issue_date' 값을 변경
for idx in indices_to_update:
    song_meta.at[idx, 'issue_date'] = new_value

In [9]:
# 알맞게 변경되었는지 확인
song_meta.iloc[[141185, 448286], :]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
141185,[GN1801],20200113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],잔잔한 바람이 불어와 (There is a gentle breeze),[GN1800],[무드홀릭 (Moodholic)],141185
448289,"[GN1003, GN1013, GN1001]",20060103,First Impressions Of Earth,312378,[101138],You Only Live Once,[GN1000],[The Strokes],448289


> 2.song_gn_dtl_gnr_basket가 비어있는 경우와 대분류 장르 GN9000 삭제

In [ ]:
song_meta['song_gn_dtl_gnr_basket_cnt'] = song_meta['song_gn_dtl_gnr_basket'].apply(len)
song_meta[song_meta['song_gn_dtl_gnr_basket_cnt']==0]

In [11]:
# 소분류 장르가 없는 song id 담아두기
song_gnr_empty = list(song_meta[song_meta['song_gn_dtl_gnr_basket_cnt'] == 0]['id'].values) # 4037개
# 장르정보가 GN9000인 song id 담아두기
song_gnr_GN9000 = list(song_meta[song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)]['id'].values) # 1834개

# 삭제될 song id list
to_del_song_id = set(song_gnr_empty + song_gnr_GN9000)
len(to_del_song_id) # 4040개

4040

In [12]:
song_meta = song_meta[song_meta['song_gn_dtl_gnr_basket_cnt'] != 0] # 소분류 장르가 없는 song 4037개 삭제
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703948 entries, 0 to 707988
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   song_gn_dtl_gnr_basket      703948 non-null  object
 1   issue_date                  703948 non-null  int64 
 2   album_name                  703948 non-null  object
 3   album_id                    703948 non-null  int64 
 4   artist_id_basket            703948 non-null  object
 5   song_name                   703948 non-null  object
 6   song_gn_gnr_basket          703948 non-null  object
 7   artist_name_basket          703948 non-null  object
 8   id                          703948 non-null  int64 
 9   song_gn_dtl_gnr_basket_cnt  703948 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 59.1+ MB


In [13]:
song_meta = song_meta[~song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)] # GN9000가 있는 행 3개삭제 (GN9000이 대분류 장르인 경우, 대부분 소분류 정보가 없어서 여기서는 3개만 삭제됨)
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703945 entries, 0 to 707988
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   song_gn_dtl_gnr_basket      703945 non-null  object
 1   issue_date                  703945 non-null  int64 
 2   album_name                  703945 non-null  object
 3   album_id                    703945 non-null  int64 
 4   artist_id_basket            703945 non-null  object
 5   song_name                   703945 non-null  object
 6   song_gn_gnr_basket          703945 non-null  object
 7   artist_name_basket          703945 non-null  object
 8   id                          703945 non-null  int64 
 9   song_gn_dtl_gnr_basket_cnt  703945 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 59.1+ MB


In [14]:
# 장르 정보가 없는 행이 없는지 확인
song_meta[song_meta['song_gn_dtl_gnr_basket_cnt']==0]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_gn_dtl_gnr_basket_cnt


In [15]:
# GN9000가 삭제되었는지 확인
song_meta[song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_gn_dtl_gnr_basket_cnt


> 3. 대분류 장르명과 song_meta merge

In [ ]:
song_meta_ex = song_meta.explode('song_gn_gnr_basket') # 대분류 장르 한줄에 하나씩 넣기
song_meta_ex.head()

In [ ]:
# 대분류 장르와 song_meta merge
song_meta_merge = pd.merge(song_meta_ex, gnr_big, how='left', left_on='song_gn_gnr_basket', right_on='gnr_code')
song_meta_merge.drop('gnr_code', axis=1, inplace=True)
song_meta_merge.head()

In [18]:
song_meta_gnr_code = song_meta_merge.groupby('id')['song_gn_gnr_basket'].apply(list).reset_index() # 하나의 song에 대해 하나의 대분류 장르코드 list로 만들기
song_meta_gnr_name = song_meta_merge.groupby('id')['gnr_name'].apply(list).reset_index() # 하나의 song에 대해 하나의 대분류 장르명 list로 만들기

In [19]:
song_meta_f = pd.merge(song_meta_merge, song_meta_gnr_code, how='left', on='id') # 장르코드 list와 merge
song_meta_f = pd.merge(song_meta_f, song_meta_gnr_name, how='left', on='id') # 장르명 list와 merge

In [ ]:
# 필요한 컬럼만 남기기
song_meta_f = song_meta_f[['song_gn_dtl_gnr_basket', 'issue_date', 'album_name', 'album_id', 'artist_id_basket', 'song_name', 'song_gn_gnr_basket_y', 'gnr_name_y', 'artist_name_basket', 'id']]
# 컬럼명 수정
song_meta_f.rename(columns={'song_gn_gnr_basket_y':'song_gn_gnr_basket', 'gnr_name_y':'gnr_name'}, inplace=True)

song_meta_f.head()

In [21]:
song_meta_cleaned = song_meta_f.drop_duplicates(subset='id', keep='first') # id가 중복인 행에 대해 하나의 id행만 남기기

In [22]:
song_meta_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703945 entries, 0 to 799765
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   song_gn_dtl_gnr_basket  703945 non-null  object
 1   issue_date              703945 non-null  int64 
 2   album_name              703945 non-null  object
 3   album_id                703945 non-null  int64 
 4   artist_id_basket        703945 non-null  object
 5   song_name               703945 non-null  object
 6   song_gn_gnr_basket      703945 non-null  object
 7   gnr_name                703945 non-null  object
 8   artist_name_basket      703945 non-null  object
 9   id                      703945 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 59.1+ MB


In [23]:
# song_meta 저장
song_meta_cleaned.to_json('../0_data/orgi_song_meta.json', orient='records')

> 4. song_meta DB용으로 만들기

In [24]:
song_meta_db = song_meta_cleaned[['id', 'song_name', 'artist_name_basket', 'gnr_name']]
song_meta_db.rename(columns={'id':'song_id', 'artist_name_basket':'artist_name_lst', 'gnr_name':'song_gnr_lst'}, inplace=True)

C:\Users\cg063\AppData\Local\Temp\ipykernel_14460\593262896.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_meta_db.rename(columns={'id':'song_id', 'artist_name_basket':'artist_name_lst', 'gnr_name':'song_gnr_lst'}, inplace=True)


In [ ]:
song_meta_db.head()

In [26]:
# db_song_meta 저장
song_meta_db.to_json('../0_data/db_song_meta.json', orient='records')

### 2. playlists

> 0. 장르정보, GN9000대분류 장르였던 노래들 삭제

In [27]:
# song_meta에서 삭제한 to_del_song_id들을 playlists에서도 삭제
playlists['songs'] = playlists['songs'].apply(lambda x: [song for song in x if song not in to_del_song_id])

In [28]:
# songs가 비어 있는 행이 있는지 확인 -> 10개(아기 클래식, 뮤지컬 동화, 자장가 등)
playlists[playlists['songs'].apply(lambda x: len(x) == 0)] 

# 비어 있는 행은 삭제
playlists = playlists[playlists['songs'].apply(lambda x: len(x) != 0)] 

In [29]:
playlists.info() # 10행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115061 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          115061 non-null  object
 1   id            115061 non-null  int64 
 2   plylst_title  115061 non-null  object
 3   songs         115061 non-null  object
 4   like_cnt      115061 non-null  int64 
 5   updt_date     115061 non-null  object
dtypes: int64(2), object(4)
memory usage: 6.1+ MB


> 1. 타이틀, 태그 불용어 처리

- 타이틀 특수문자 제거

In [30]:
# 이모지 제거 함수
def replace_emoji(inputString):
    return core.replace_emoji(inputString, replace='')

# 특수문자 제거 함수
def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^●○.@*\"※~ㆍ!<>$♥❤♡☆★♪♬♩¶♨①๑｡＊②ఇ¥◈ ͡✔ ͡° ͜ʖ▶◀◎ღ✿ﾉ ;\\\_■『│』ᕷⓗ‘|\(\)\[\]`\'…》\”\“\’·]',' ', inputString)
  return text_rmv

In [31]:
# # 이모지 제거 함수 적용
playlists['plylst_title'] = playlists['plylst_title'].apply(replace_emoji)

# 특수문자 제거 함수 적용
playlists['plylst_title'] = playlists['plylst_title'].apply(clean_text)

In [32]:
# 양 옆 공백, %, & 제거

playlists['plylst_title'] = playlists['plylst_title'].str.strip()
playlists['plylst_title'] = playlists['plylst_title'].str.strip('%')
playlists['plylst_title'] = playlists['plylst_title'].str.strip('&')

In [33]:
playlists[playlists['id'].isin([22525, 143039])]

,tags,id,plylst_title,songs,like_cnt,updt_date
56134,[흥폭발],22525,내적댄스 터져베이베 아이돌 노래노래,"[108003, 594076, 297411, 97189, 35713, 445680,...",41,2018-06-15 19:24:26.000
59162,"[가을, 설렘, 사랑]",143039,가을이다 o 곧 겨울이다 T T,"[13142, 5049, 503816, 70166, 578529, 336829, 2...",1,2018-07-31 10:55:04.000


- 태그 특수문자 제거

In [34]:
# 특수문자 '_'가 있는 행 찾기
# '_'가 있는 행 찾기
rows_with_underscore = playlists['tags'][playlists['tags'].apply(lambda x: any('_' in tag for tag in x))]
rows_with_underscore

148                            [비오는날, 비_오는_날, 휴식, 비_오는날, 비]
285                                [상쾌해지는, 호불호없는, 모두가_흥얼흥얼]
364        [추천, 일렉트로니카, 5월_4주차, 주간, 최신, 전지음악, 인기, 주간일렉트로니카]
409                      [들었으면, 인디음악, 한번쯤, 이미지는_직접찍은사진, 인디]
522       [힙합, label_crew, 랩, 국내힙합, 트렌드, 쇼미더머니, 스웨그, aom...
                                ...                        
114625                                   [칠_Chill, 설레는_달콤함]
114647                     [우울, 위로, 위로가_필요할_때, 슬픔, 이별, 힘든날]
114688    [어쿠스틱, 혼자, 발라드, 우울, 분위기, 멜로디, 띵곡들, 조용한, 자기_전, 인디]
114703    [패션위크, 뉴욕, 도시, 갈라쇼음악, 트렌디, 팝, 편집샵, i_know_it, ...
115068               [담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]
Name: tags, Length: 2252, dtype: object

In [35]:
# 특수 문자, 빈 태그를 제거하는 함수 정의
def remove_special_characters(tag_list):
    cleaned_tags = []
    for tag in tag_list:
        cleaned_tag = re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z\s]', '', tag).replace(" ", "")
        if cleaned_tag:
            cleaned_tags.append(cleaned_tag)
    return cleaned_tags

# 함수 적용
playlists['tags'] = playlists['tags'].apply(remove_special_characters)

In [36]:
playlists[playlists['id'].isin([32533, 151357, 11343, 95032])]

,tags,id,plylst_title,songs,like_cnt,updt_date
148,"[비오는날, 비오는날, 휴식, 비오는날, 비]",32533,비오는날 감성 자극 듣기 좋은 어쿠스틱 피아노 발라드,"[133091, 527764, 468136, 260894, 22845, 148876...",14,2020-02-16 00:56:45.000
285,"[상쾌해지는, 호불호없는, 모두가흥얼흥얼]",151357,기분좋아지는 노래들,"[270061, 158766, 573179, 272062, 37027, 245028...",0,2019-11-24 05:49:55.000
56921,"[힙합, 흑인음악, 힙합엘이, 힙합추천, 틱톡, 외힙, HIPHOPLE]",95032,어떻게 떴냐고 답은 바로 TikTok,"[253222, 270262, 301533, 136007, 238347, 13384...",7,2020-01-08 11:19:11.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의이야기, 사랑, 친구]",11343,1 눈물이 앞을 가리는 나의 이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000


In [37]:
# 플레이리스트 타이틀이 1글자인 조건 만들기
playlists_title_1str = playlists[playlists['plylst_title'].str.len() == 1]
playlists_title_1str = list(playlists_title_1str['id'].values) # 24 -> 롹, 잠, 쿨, 힙 등이며 대부분 좋아요 0 또는 1
# playlists[playlists['id'] == 98628]

# 삭제
playlists = playlists[playlists['plylst_title'].str.len() != 1]

In [38]:
# 빈 타이틀 삭제
playlists = playlists[playlists['plylst_title'] != '']

# 숫자로만 구성된 타이틀 삭제
numeric_title_rows = playlists[playlists['plylst_title'].str.isnumeric()]
indices_to_remove = numeric_title_rows.index
playlists = playlists.drop(indices_to_remove)

playlists.info()


# 115061  - 114697  = 364행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114697 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          114697 non-null  object
 1   id            114697 non-null  int64 
 2   plylst_title  114697 non-null  object
 3   songs         114697 non-null  object
 4   like_cnt      114697 non-null  int64 
 5   updt_date     114697 non-null  object
dtypes: int64(2), object(4)
memory usage: 6.1+ MB


> 2. 타이틀 중복 제거

In [39]:
# 1. 플레이리스트 아이디(id)와 플레이리스트 명(plylst_title) 추출
plylst_title = playlists[['id', 'plylst_title']]

# 2. 플레이리스트 이름 별 플레이리스트 ID count 테이블 생성 : plylst_id_cnt
plylst_id_cnt = plylst_title.groupby('plylst_title').id.nunique().reset_index(name = 'mapping_plylst_cnt')

# 3. 플레이리스트 명 중복 구분 : 서로 다른 플레이리스트 아이디로 겹치는 플레이리스트 이름이 없으면 '중복 없음', 그 외 '중복 있음'
plylst_id_cnt = plylst_id_cnt.assign(
    mapping_plylst_cnt_category = pd.cut(plylst_id_cnt['mapping_plylst_cnt'], [0, 1, np.inf], labels = ['중복 없음', '중복 있음'])
)
plylst_id_cnt
# [0, 1, np.inf] : 0이상 1미만 값 = 중복없음, 1이상의 값 = 중복있음

# 4. 중복 구분 별 플레이리스트 수 count 테이블 생성 : plylst_id_cnt_division
plylst_id_cnt_division = pd.DataFrame(plylst_id_cnt.groupby('mapping_plylst_cnt_category').count())
plylst_id_cnt_division

,plylst_title,mapping_plylst_cnt
mapping_plylst_cnt_category,,
중복 없음,106515,106515
중복 있음,2722,2722


In [ ]:
# 플레이리스트 타이틀을 기준으로 merge해 중복된 플레이리스트만 확인
playlists_dup = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_dup= playlists_dup[playlists_dup['mapping_plylst_cnt_category'] == '중복 있음']
playlists_dup['song_cnt'] = playlists_dup['songs'].apply(len)
playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)
playlists_dup.head()

In [ ]:
# 같은 타이틀을 가진 플레이리스트의 노래 개수 비교
# playlists_dup에서 타이틀별로 groupby 후 song_cnt 값들을 song_cnt_nunique(새 컬럼)에 리스트 형태로 넣은 후 중복을 없애기 위해 set 진행
# 1의 의미는 노래 갯수가 중복되어 있다는 뜻이기 때문에 1의 갯수가 적은 지표를 기준으로 중복 플레이리스트 제거를 진행해야함
from collections import Counter

playlists_dup_song_cnt = playlists_dup.groupby('plylst_title')['song_cnt'].apply(list).reset_index()
playlists_dup_song_cnt['song_cnt_nunique'] = playlists_dup_song_cnt['song_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_song_cnt['song_cnt_nunique'])

In [ ]:
# 같은 타이틀을 가진 플레이리스트의 태그 개수 비교

playlists_dup_tag_cnt = playlists_dup.groupby('plylst_title')['tag_cnt'].apply(list).reset_index()
playlists_dup_tag_cnt['tag_cnt_nunique'] = playlists_dup_tag_cnt['tag_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_tag_cnt['tag_cnt_nunique'])

In [ ]:
# 같은 타이틀을 가진 플레이리스트의 좋아요 비교

playlists_dup_like_cnt = playlists_dup.groupby('plylst_title')['like_cnt'].apply(list).reset_index()
playlists_dup_like_cnt['like_cnt_nunique'] = playlists_dup_like_cnt['like_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_like_cnt['like_cnt_nunique'])

- 플레이리스트 타이틀이 중복인 플레이리스트만 따로 모으고
- 그 중복 플리모음 중 노래갯수가 가장 많은 하나만 남겨두고
- 원래 train의 중복 없음만 따로 모아 중복모음과 concat

In [ ]:
# train에 중복 갯수, 중복 여부 컬럼 추가, 플레이리스트 내 곡 갯수 추가
playlists_add_col = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_add_col['song_cnt'] = playlists_add_col['songs'].apply(lambda x : len(x))

# 매핑갯수, 타이틀, 플리 내 노래 갯수, like_cnt 내림차순 정렬
playlists_filter = playlists_add_col.sort_values(['mapping_plylst_cnt', 'plylst_title', 'song_cnt', 'like_cnt'], ascending=False)
playlists_filter

In [45]:
playlists_dup = playlists_filter[playlists_filter['mapping_plylst_cnt_category'] == '중복 있음'] # 중복 플리 모음
playlists_dup['plylst_title'].nunique() # 중복된 플레이리스트의 유니크 타이틀 갯수 : 2722

2722

In [ ]:
playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)
playlists_dup.head()

In [ ]:
# playlists_dup 중 각 플리 제목 별 노래갯수가 가장 많은 하나만 남긴다
playlists_dup_filter = pd.DataFrame(playlists_dup[playlists_dup.groupby('plylst_title')['song_cnt'].transform(max) == playlists_dup['song_cnt']]) # 3572 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['like_cnt'].transform(max) == playlists_dup_filter['like_cnt']]) # 2928 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['tag_cnt'].transform(max) == playlists_dup_filter['tag_cnt']]) # 2909 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['updt_date'].transform(max) == playlists_dup_filter['updt_date']]) # 2748 
playlists_dup_filter

In [48]:
playlists_dup_filter[playlists_dup_filter['plylst_title'] == 'qqqa'] # 필터링 되지 않은 중복 타이틀은 플레이리스트 자체가 같음

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
50436,[여름],7524,qqqa,"[522825, 455668, 549178, 407828, 343974]",1,2015-11-04 17:51:14.000,3,중복 있음,5,1
50437,[여름],43031,qqqa,"[522825, 455668, 549178, 407828, 343974]",1,2015-11-04 17:51:14.000,3,중복 있음,5,1


In [49]:
# 타이틀이 같은 플레이리스트 중 하나만 남긴다
playlists_dup_filter = pd.DataFrame(playlists_dup_filter.groupby('plylst_title', group_keys=False).apply(lambda x: x.loc[x['song_cnt'].idxmax()]))
playlists_dup_filter = playlists_dup_filter.reset_index(drop=True) # 2722

In [50]:
# playlists_dup_filter 중 중복 여부 확인

# 1. 플레이리스트 아이디(id)와 플레이리스트 명(plylst_title) 추출
plylst_title_2 = playlists_dup_filter[['id', 'plylst_title']]

# 2. 플레이리스트 이름 별 플레이리스트 ID count 테이블 생성 : plylst_id_cnt
plylst_id_cnt_2 = plylst_title_2.groupby('plylst_title').id.nunique().reset_index(name = 'mapping_plylst_cnt')

# 3. 플레이리스트 명 중복 구분 : 서로 다른 플레이리스트 아이디로 겹치는 플레이리스트 이름이 없으면 '중복 없음', 그 외 '중복 있음'
plylst_id_cnt_2 = plylst_id_cnt_2.assign(
    mapping_plylst_cnt_category = pd.cut(plylst_id_cnt_2['mapping_plylst_cnt'], [0, 1, np.inf], labels = ['중복 없음', '중복 있음'])
)
plylst_id_cnt_2
# [0, 1, np.inf] : 0이상 1미만 값 = 중복없음, 1이상의 값 = 중복있음

# 4. 중복 구분 별 플레이리스트 수 count 테이블 생성 : plylst_id_cnt_division
plylst_id_cnt_division_2 = pd.DataFrame(plylst_id_cnt_2.groupby('mapping_plylst_cnt_category').count())
plylst_id_cnt_division_2

,plylst_title,mapping_plylst_cnt
mapping_plylst_cnt_category,,
중복 없음,2722,2722
중복 있음,0,0


In [ ]:
# 중복 없음 파일만 만들기
playlists_no_dup = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_no_dup = playlists_no_dup[playlists_no_dup['mapping_plylst_cnt_category'] == '중복 없음']
playlists_no_dup # 106515행

In [52]:
playlists_dup_proc = pd.concat([playlists_dup_filter, playlists_no_dup], axis=0)
playlists_dup_proc = playlists_dup_proc.iloc[:, :6]
playlists_dup_proc.info()

# 2722+106515 =109237
# 114697 - 109237 = 타이틀 중복 제거로 5460행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109237 entries, 0 to 114696
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          109237 non-null  object
 1   id            109237 non-null  int64 
 2   plylst_title  109237 non-null  object
 3   songs         109237 non-null  object
 4   like_cnt      109237 non-null  int64 
 5   updt_date     109237 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.8+ MB


In [73]:
playlists_copy = playlists_dup_proc.copy()

In [74]:
playlists_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109237 entries, 0 to 114696
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          109237 non-null  object
 1   id            109237 non-null  int64 
 2   plylst_title  109237 non-null  object
 3   songs         109237 non-null  object
 4   like_cnt      109237 non-null  int64 
 5   updt_date     109237 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.8+ MB


> 3. 플레이리스트 별 노래 개수, 좋아요 개수 이상치 행 날리기

- 노래 개수 5~104개 가진 플레이리스트만 남기기 <- boxplot IQR 범위 내

In [75]:
playlists_copy['songs_cnt'] = playlists_copy['songs'].apply(len)
playlists_copy['songs_cnt'].describe()

count    109237.000000
mean         45.949321
std          44.216494
min           1.000000
25%          19.000000
50%          29.000000
75%          53.000000
max         200.000000
Name: songs_cnt, dtype: float64

In [76]:
Q1 = 19
Q3 = 53
IQR = Q3 - Q1

print(Q1-(1.5*IQR))
print(Q3+(1.5*IQR))

-32.0
104.0


In [77]:
playlists_copy['songs_cnt'] = playlists_copy['songs'].apply(lambda x : len(x))
playlists_song_cut = playlists_copy[(playlists_copy['songs_cnt'] >= 5) & (playlists_copy['songs_cnt'] < 105) ]  # 5~104개만 남기기
playlists_song_cut.info() # 109237-98973 = 10264행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98973 entries, 0 to 114695
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          98973 non-null  object
 1   id            98973 non-null  int64 
 2   plylst_title  98973 non-null  object
 3   songs         98973 non-null  object
 4   like_cnt      98973 non-null  int64 
 5   updt_date     98973 non-null  object
 6   songs_cnt     98973 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 6.0+ MB


- like_cnt 6개 이하 삭제 (like_cnt 50% = 7)

In [78]:
playlists_song_cut['like_cnt'].describe()

count    98973.000000
mean        77.190284
std        437.540113
min          0.000000
25%          2.000000
50%          7.000000
75%         22.000000
max      53211.000000
Name: like_cnt, dtype: float64

In [79]:
Q1 = 2
Q3 = 22
IQR = Q3 - Q1

print(Q1-(1.5*IQR))
print(Q3+(1.5*IQR))

-28.0
52.0


In [80]:
playlists_song_like_cut = playlists_song_cut[playlists_song_cut['like_cnt'] >= 7]

In [81]:
playlists_song_like_cut = playlists_song_like_cut.iloc[:, :6]
playlists_song_like_cut.info()# 98973 - 51916 = 47057행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51916 entries, 8 to 114693
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          51916 non-null  object
 1   id            51916 non-null  int64 
 2   plylst_title  51916 non-null  object
 3   songs         51916 non-null  object
 4   like_cnt      51916 non-null  int64 
 5   updt_date     51916 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.8+ MB


In [82]:
playlists_copy_cnt = playlists_song_like_cut.copy()
playlists_copy_cnt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51916 entries, 8 to 114693
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          51916 non-null  object
 1   id            51916 non-null  int64 
 2   plylst_title  51916 non-null  object
 3   songs         51916 non-null  object
 4   like_cnt      51916 non-null  int64 
 5   updt_date     51916 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.8+ MB


> 4. song, 태그 빈도수에 따라 리스트에서 빼기 -> 리스트가 비워져있으면 행 날리기

- 노래 빈도수와 좋아요 상관관계

In [83]:
# 노래 별 좋아요 수 확인
ply_song_like = playlists_copy_cnt.explode('songs').groupby('songs').sum()['like_cnt']
ply_song_like = pd.DataFrame(ply_song_like).reset_index()
ply_song_like

C:\Users\cg063\AppData\Local\Temp\ipykernel_14460\3556108467.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ply_song_like = playlists_copy_cnt.explode('songs').groupby('songs').sum()['like_cnt']


,songs,like_cnt
0,0,27
1,3,1650
2,4,8
3,5,382
4,6,24
...,...,...
396450,707978,642
396451,707979,72
396452,707980,8
396453,707985,7


In [84]:
# 노래 사용 빈도수 확인
ply_song_freq = playlists_copy_cnt.explode('songs').groupby('songs').count()['id']
ply_song_freq = pd.DataFrame(ply_song_freq).reset_index()
ply_song_freq.rename(columns={'id':'freq'}, inplace=True)
ply_song_freq

,songs,freq
0,0,1
1,3,5
2,4,1
3,5,2
4,6,1
...,...,...
396450,707978,2
396451,707979,2
396452,707980,1
396453,707985,1


In [85]:
playlists_copy_cnt[playlists_copy_cnt['songs'].apply(lambda x: 707978 in x)] # 2번 사용됨

,tags,id,plylst_title,songs,like_cnt,updt_date
17738,"[추울때, 어쿠스틱, 잔잔한, 새벽감성, 따뜻한, 티타임, 밤공기, 서정적인, 독서감상]",136386,독서할때 들으면 좋은 서정적이고 따듯한 팝송,"[562883, 696713, 537409, 464248, 430558, 59447...",19,2019-10-30 08:09:50.000
30493,"[감성, 겨울저녁, 새벽, 해외팝, 휴식, 팝]",11170,다가오는 겨울 새벽 내내 틀어놓고 싶은 노래,"[314302, 316407, 553161, 599409, 671675, 17680...",623,2019-11-25 09:55:34.000


In [86]:
ply_corr_song = pd.merge(ply_song_like, ply_song_freq, how='left', on='songs')
ply_corr_song

,songs,like_cnt,freq
0,0,27,1
1,3,1650,5
2,4,8,1
3,5,382,2
4,6,24,1
...,...,...,...
396450,707978,642,2
396451,707979,72,2
396452,707980,8,1
396453,707985,7,1


In [87]:
ply_corr_song.describe()

,songs,like_cnt,freq
count,396455.000000,396455.000000,396455.000000
mean,353788.525855,937.842930,4.929281
std,204261.411905,3425.699171,15.894259
min,0.000000,7.000000,1.000000
25%,176795.000000,23.000000,1.000000
50%,353911.000000,101.000000,1.000000
75%,530728.500000,534.000000,3.000000
max,707986.000000,154389.000000,894.000000


In [88]:
ply_corr_song.corr()

,songs,like_cnt,freq
songs,1.000000,-0.001523,-0.000072
like_cnt,-0.001523,1.000000,0.785140
freq,-0.000072,0.785140,1.000000


- 1회 이하 사용된 song, songs 리스트에서 빼기

In [89]:
# song usage count 계산
songs_counts = playlists_copy_cnt.explode('songs')['songs'].value_counts(ascending=True)

# 1번 이하 사용된 song
less_than_1_songs = list(songs_counts[songs_counts == 1].index) # 211447

In [90]:
len(less_than_1_songs)

211447

In [91]:
pd.DataFrame(playlists_copy_cnt.explode('songs')['songs']).nunique() # 자르기 전 유니크 곡 : 396455

songs    396455
dtype: int64

In [95]:
# 1번 이하 사용된 songs 삭제

from tqdm import tqdm
tqdm.pandas() 
playlists_copy_cnt['songs'] = playlists_copy_cnt['songs'].progress_apply(lambda songs: list(set(songs).difference(less_than_1_songs)))

  0%|          | 0/51916 [00:00<?, ?it/s]

100%|██████████| 51916/51916 [05:39<00:00, 153.06it/s]


In [ ]:
# songs가 비어있지 않은 행만 남김
playlists_song_cut = playlists_copy_cnt[playlists_copy_cnt['songs'].apply(lambda x: len(x) > 0)]
playlists_song_cut.head()

In [97]:
playlists_song_cut['song_cnt'] = playlists_song_cut['songs'].apply(len)
playlists_song_cut['song_cnt'].unique()

C:\Users\cg063\AppData\Local\Temp\ipykernel_14460\558675112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlists_song_cut['song_cnt'] = playlists_song_cut['songs'].apply(len)


array([ 23,  34,  12,  50,  58,  27,  14,  18,   8,  16,  30,  15,   7,
        17,  33,  29,  70,  26,  66,  24,  25,  19,  71,  32,  98,  72,
        82,  85,  87,  48,  35,  91,   5,  10,  31,  96,  53,  37,  11,
        28,  81,  21,  69,  51,  13,  56,  43,  41,  73,  76,   9,  75,
        64,  61,  40,  54,  22,  62,  42,  49,  67,  46,  65,  20,  36,
        74,  55,  86,  80,  78,  94,  52, 103,  44,  60,  77,   6,  84,
        68,  47,  38, 100,  57,  45,  89,  83,  99,   3,  92,  39,  95,
        90,  63,  59,  97,  88, 102,  79,   1, 104,   4,   2,  93, 101],
      dtype=int64)

In [98]:
playlists_song_cut.explode('songs')['songs'].value_counts(ascending=True) # 1회 이하 사용된 song들이 알맞게 제거 되었는지 확인

49730       2
81475       2
266849      2
98213       2
13368       2
         ... 
418935    593
144663    646
366786    684
116573    754
205179    894
Name: songs, Length: 185008, dtype: int64

In [99]:
pd.DataFrame(playlists_song_cut.explode('songs')['songs']).nunique() # 390852개 -> 185008개로 삭제

songs    185008
dtype: int64

In [100]:
playlists_song_cut = playlists_song_cut.iloc[:, :6]
playlists_song_cut.info()

# 51916 - 51724 = 192행 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51724 entries, 8 to 114693
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          51724 non-null  object
 1   id            51724 non-null  int64 
 2   plylst_title  51724 non-null  object
 3   songs         51724 non-null  object
 4   like_cnt      51724 non-null  int64 
 5   updt_date     51724 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.8+ MB


- 태그 빈도수와 좋아요 상관관계

In [101]:
pd.DataFrame(playlists_song_cut.explode('tags')['tags']).nunique() # 19393개

tags    19393
dtype: int64

In [102]:
# 태그 별 좋아요 수 확인
ply_tag_like = playlists_song_cut.explode('tags').groupby('tags').sum()['like_cnt']
ply_tag_like = pd.DataFrame(ply_tag_like).reset_index()
ply_tag_like

C:\Users\cg063\AppData\Local\Temp\ipykernel_14460\573314142.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ply_tag_like = playlists_song_cut.explode('tags').groupby('tags').sum()['like_cnt']


,tags,like_cnt
0,00,158
1,007,100
2,007시리즈,65
3,00s,10
4,00년,19
...,...,...
19388,힙해,2365
19389,힙힙힙,13
19390,힛뎀포크,41
19391,힛뎀폭,41


In [103]:
# 노래 사용 빈도수 확인
ply_tag_freq = playlists_song_cut.explode('tags').groupby('tags').count()['id']
ply_tag_freq = pd.DataFrame(ply_tag_freq).reset_index()
ply_tag_freq.rename(columns={'id':'freq'}, inplace=True)
ply_tag_freq

,tags,freq
0,00,3
1,007,2
2,007시리즈,2
3,00s,1
4,00년,1
...,...,...
19388,힙해,4
19389,힙힙힙,1
19390,힛뎀포크,1
19391,힛뎀폭,1


In [104]:
playlists_song_cut[playlists_song_cut['tags'].apply(lambda x: '007' in x)] # 2번 사용됨

,tags,id,plylst_title,songs,like_cnt,updt_date
692,"[액션히어로, 스파이영화, 본시리즈, OST, 007, 미션임파서블, 긴장감, 장르...",46937,긴장감 넘치는 액션과 사운드 첩보 스파이물 대표 OST,"[634628, 657608, 359929, 496978, 122681, 83546...",75,2017-07-13 10:23:13.000
75148,"[아델, 샘스미스, 제임스본드, 빌리아일리쉬, 007, 주제곡, 영화음악]",45506,007 제임스 본드 테마 음악 스페셜,"[594502, 47303, 568983, 515742, 333856, 291812...",25,2020-02-18 12:17:09.000


In [105]:
ply_corr_tag = pd.merge(ply_tag_like, ply_tag_freq, how='left', on='tags')
ply_corr_tag

,tags,like_cnt,freq
0,00,158,3
1,007,100,2
2,007시리즈,65,2
3,00s,10,1
4,00년,19,1
...,...,...,...
19388,힙해,2365,4
19389,힙힙힙,13,1
19390,힛뎀포크,41,1
19391,힛뎀폭,41,1


In [106]:
ply_corr_tag.describe()

,like_cnt,freq
count,1.939300e+04,19393.000000
mean,2.076583e+03,12.201722
std,2.572613e+04,135.322666
min,7.000000e+00,1.000000
25%,1.700000e+01,1.000000
50%,4.900000e+01,1.000000
75%,2.570000e+02,3.000000
max,1.514935e+06,7300.000000


In [107]:
# tag usage count 계산
tag_counts = playlists_song_cut.explode('tags')['tags'].value_counts(ascending=True)

# 1번 사용된 태그
less_than_1_tags = list(tag_counts[tag_counts == 1].index) # 11743

In [108]:
len(less_than_1_tags)

11743

In [109]:
from tqdm import tqdm
tqdm.pandas()

playlists_song_cut['tags'] = playlists_song_cut['tags'].progress_apply(lambda tags: list(set(tags).difference(less_than_1_tags)))

100%|██████████| 51724/51724 [00:22<00:00, 2266.83it/s]


In [ ]:
# tags가 비어있지 않은 행만 남김
playlists_song_cut = playlists_song_cut[playlists_song_cut['tags'].apply(lambda x: len(x) > 0)]
playlists_song_cut.head()

In [111]:
playlists_song_cut['tag_cnt'] = playlists_song_cut['tags'].apply(len)
playlists_song_cut['tag_cnt'].unique()

array([ 2,  1,  5,  4,  3,  6, 10,  9,  7,  8, 11], dtype=int64)

In [112]:
playlists_song_cut.explode('tags')['tags'].value_counts(ascending=True)

YESorYES        1
TakeOut         1
1일1클래식1기쁨       1
커피숍노래           2
월간차트            2
             ... 
잔잔한          4287
휴식           4696
감성           4825
드라이브         4835
기분전환         7299
Name: tags, Length: 7650, dtype: int64

In [113]:
playlists_song_tag_cut = playlists_song_cut.iloc[:, :6]
playlists_song_tag_cut.info()

# 51724 - 51479   = 245행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51479 entries, 8 to 114693
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          51479 non-null  object
 1   id            51479 non-null  int64 
 2   plylst_title  51479 non-null  object
 3   songs         51479 non-null  object
 4   like_cnt      51479 non-null  int64 
 5   updt_date     51479 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.7+ MB


In [114]:
pd.DataFrame(playlists_song_tag_cut.explode('tags')['tags']).nunique() # 19393개 -> 7650개

tags    7650
dtype: int64

In [115]:
pd.DataFrame(playlists_song_tag_cut.explode('songs')['songs']).nunique() # 185004

songs    185004
dtype: int64

In [116]:
185004 + 7650

192654

> 정리

=> song_meta : 날짜 수정, 장르 정보 없는 노래 제거  
=> playlists  
    - 타이틀 & 태그 불용어 삭제 : 특수문자, 빈 타이틀, 모두 숫자인 경우, 1글자인 경우 포함  
    - 타이틀 중복 제거 : 타이틀이 완벽하게 같을 경우 '1_노래 갯수 많은 순서 -> 2_좋아요 갯수 많은 순서 -> 3_태그 갯수 많은 순서 -> 4_최신 업데이트' 기준으로 1개씩 골라냄 (4가지를 했는데도 같은건 완벽히 같았기에 하나만 남김)  
    - 중복 제거 후 song_cnt describe 해 5~104개 남기기, like_cnt describe 7개 이상인 플레이리스트(50%=7)
    - 1번 사용된 tag, 1번 이하 사용된song 제거  
  
  
=> 115071(원래 playlist) -51479(전처리 후) = 63592행 삭제

> 데이터 저장

- 최종 전처리 파일은 ../0_data/tag1,song1_playlists.json 으로 저장

In [117]:
# playlists 저장
playlists_song_tag_cut.to_json('../0_data/tag1,song1_playlists.json', orient='records')

---